In [1]:
# Skriptas apjungtų duomenų Zive ir Mit2zive testavimui
# Suranda užduotas anotacijas ir sukuria grafinius vaizdus su surastais pūpsniais 
# ir jų kaimynais kairėje ir dešinėje 
# 
# Pritaikytas tiek MIT2ZIVE, tiek Zive duomenims

import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import sys, json
from zive_util_vu import create_dir, get_rec_file_name, get_recId
from zive_util_vu import get_seq_start_end, read_rec, read_rec_attrib, split_SubjCode
from zive_cnn_fda_vu_v3_micro import read_RR_arr_from_signal, read_seq_from_signal

# import math
# import random
# import scipy.signal
# import skfda
# from skfda import FDataGrid
# from skfda.preprocessing.smoothing import BasisSmoother
# from skfda.representation.basis import BSpline, Fourier

 # Pasiruošimas

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# ++++++++++++++++++++++  variantas su visais duomenimis ++++++++++++++++++++++++++++++
# Bendras duomenų aplankas

if OS == 'Windows':
    Duomenu_aplankas = 'D:\\DI'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI'   # arba variantas: UBUNTU

# Vietinės talpyklos aplankas
db_folder = 'DUOM_2022_RUDUO_2'

# Nuoroda į aplanką su EKG duomenų rinkiniu
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(db_path, 'records_npy')


# +++++++++++++++++++++++++ variantas su testiniais duomenimis +++++++++++++++++++++++++++++

# Bendras duomenų aplankas

# Duomenu_aplankas = Path.cwd()
# https://miguendes.me/how-to-find-the-current-working-directory-in-python

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
# rec_dir = Path(Duomenu_aplankas, 'PVZ_SKIRTAS_VU_RUDUO')

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


# Failas pūpsnių klasių formavimui: tinka tiek zive tiek mit2zive duomenims
all_beats = {'N':0,'R':0, 'L':0, 'e':0, 'j':0, 'A':1,'a':1, 'J':1, 'S':1, 'V':2, 'E':2, 'F':3, 'U':3, 'Q':3}

# Diskretizavimo dažnis:
fs = 200

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', all_beats)


OS in my system :  win32

Bendras duomenų aplankas:  D:\DI
Aplankas su originaliais EKG įrašais ir anotacijomis (.json)  D:\DI\DUOM_2022_RUDUO_2\records_npy
Diskretizavimo dažnis:  200
Klasifikavimo schema: {'N': 0, 'R': 0, 'L': 0, 'e': 0, 'j': 0, 'A': 1, 'a': 1, 'J': 1, 'S': 1, 'V': 2, 'E': 2, 'F': 3, 'U': 3, 'Q': 3}


In [2]:
def get_symbol_list(atr_symbol, atr_sample, seq_start, seq_end):
    # Surenkame išpjautos EKG sekos anotacijas ir jų indeksus sekoje
    # ir patalpiname sąraše.
    beat_locs = []
    beat_symbols = []

    for i in range(atr_sample.shape[0]):
        if atr_sample[i] > seq_start and atr_sample[i] < seq_end:
            beat_symbols.append(atr_symbol[i])
            beat_locs.append(atr_sample[i]-seq_start)   
            # beat_locs.append(atr_samples[i])   

    return (beat_symbols,beat_locs)

def create_selected_beats_df(SubjCode, atr_symbol, atr_sample, wl_side_beats, wr_side_beats, side_ext, beat_label):
# Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    
    df_beats_attr = pd.DataFrame({'SubjCode': pd.Series(dtype='int'),
                   'from': pd.Series(dtype='int'),
                   'sample': pd.Series(dtype='int'),
                   'until': pd.Series(dtype='int'),
                   'symbol': pd.Series(dtype='str')})
    
    for i, i_sample in enumerate(atr_sample):
        if  (atr_symbol[i] == beat_label):
            
            from_at = -1
            i_from = i - wl_side_beats
            if (i_from > 0):
                from_at = atr_sample[i_from] - side_ext

            until_at = -1
            i_until = i + wr_side_beats
            if (i_until < len(atr_sample)): # patikrinti dėl =
                until_at = atr_sample[i_until] + side_ext

            if (from_at >= 0 and until_at > 0 ):
                beats_attr = {'SubjCode':int(SubjCode), 'from':int(from_at),  'sample':int(i_sample), 'until':int(until_at),'symbol':str(atr_symbol[i])}

                # Surenkame visus atvejus į dataframe
                df_new_row = pd.DataFrame([beats_attr])
                df_beats_attr = pd.concat([df_beats_attr, df_new_row])

    # Pernumeruojame indeksus, kad būtų nuo 0 iš eilės
    df_beats_attr.reset_index(inplace = True, drop = True)
    return(df_beats_attr)


def show_selected_beats(signal, atr_symbol, atr_sample, df_beats_attr, images_dir):
    
    for idx, row in df_beats_attr.iterrows():
        # Formuojami grafiniai vaizdai
        seq_start = row['from']
        seq_end = row['until']
        beat_rpeak = row['sample']
        beat_symbol = row['symbol']
        sequence = signal[seq_start:seq_end] 

        # # suformuojame anotacijų žymes
        beat_symbols, beat_locs = get_symbol_list(atr_symbol, atr_sample, seq_start, seq_end)
        # print(beat_locs)
        # print(beat_symbols)

        # deltax ir deltay simbolių pozicijų koregavimui
        min = np.amin(sequence)
        max = np.amax(sequence)
        deltay = (max - min)/20
        deltax = len(sequence)/100

        # suformuojame vaizdą
        fig = plt.figure(facecolor=(1, 1, 1), figsize=(24,6))
        ax = plt.gca()
        x = np.arange(0, len(sequence), 1)
        ax.plot(x, sequence, color="#6c3376", linewidth=2)
        for i in range(len(beat_locs)):
            ax.annotate(beat_symbols[i], (beat_locs[i] - deltax,sequence[beat_locs[i]] + deltay))
        ax.set_ylim([min, max+2*deltay])
    
        SubjCode = row['SubjCode']
        seq_name = str(SubjCode) + '_' + str(beat_rpeak)
        file_name_orig = get_rec_file_name(rec_dir, SubjCode)
               
        # suformuosime koreguotą failo vardą
        if (file_name_orig == None):
            file_name = seq_name + '_' + beat_symbol + ".png"
        else:
            file_name = str(file_name_orig) + '_' + seq_name + '_' + beat_symbol + ".png"

        beat_folder = beat_symbol + '_' + 'beats'
        
        beat_images_dir = Path(images_dir, beat_folder)
        if (not Path(beat_images_dir).is_dir()):
            create_dir(beat_images_dir) 
        file_path = Path(beat_images_dir, file_name)
        print('file_path: ', file_path)   
        
        ax.set_title(file_name)
        plt.savefig(file_path, bbox_inches='tight', pad_inches = 0.2)
        plt.close()

# Ciklo per pacientų įrašus pabaiga



In [5]:
# Suformuojame pūpsnių ir jų aplinkos atributų dataframe, suformuojame vaizdus
#
# Užduodame pūpsnių vardus
lst_beat_labels = ['S']

# Variantas: testinė imtis  
file_path = Path(rec_dir, 'test_subjcode_lst_z.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

# Nagrinėjamų pacientų kodai
SubjCodes = [10010,10020,10021]

# Bendras aplankas vaizdams
images_folder = 'saved_images_of_selected_annotations'

# Užduodame, kiek sekų vaizdų iš kiekvienos klasės įrašysime į diską
img_max = 20

# /////////////////////////////////////////////////////////////////

print("\nBendras aplankas vaizdams:", images_folder)
print("Max sekų vaizdų skaičius iš kiekvienos klasės:", img_max)
print("Ieškomi pūpsniai:", lst_beat_labels)
print("Nagrinėjamų įrašų sąrašas:", SubjCodes)

if (img_max == 0):
    sys.exit()

# sukuriame bendrą aplanką vaizdams
images_dir = Path(rec_dir, images_folder)
if (not Path(images_dir).is_dir()):
    create_dir(images_dir)

# nurodome, kiek pūpsnių naudojama prieš vaizduojamą pūpsnį ir po
wl_side_beats, wr_side_beats = 5, 5

# užduodame, kiek reikšmių vaizduosime prieš R dantelį ir po
side_ext = 100


#  # Ciklas per pacientų įrašus
for SubjCode in SubjCodes:

    # Surandame ir išvedame paciento ir įrašo kodus, EKG reikšmių kiekį
    userNr, recNr = split_SubjCode(SubjCode)
    userId, recId, file_name = get_recId(rec_dir, userNr, recNr)
    # file_name_orig = get_rec_file_name(rec_dir, SubjCode)
    if (userId == None):
        print(f"\nSubjCode: {SubjCode}")
    else:
        print(f"\nSubjCode: {SubjCode}  file_name: {file_name} userId: {userId} recId: {recId}")

    # Nuskaitome EKG įrašą (npy formatu) - kaip surasti signal_length, nenuskaičius npy failo
    signal = read_rec(rec_dir, SubjCode)
    signal_length = signal.shape[0]

    # Nuskaitome iš json failo EKG įrašo rpeaks indeksus ir atitinkamas anotacijas
    # (rpeaks: "sampleIndex" ir "annotationValue") 
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)
    # print(atr_sample[:10])
    # print(atr_symbol[:10])

    # Iš anotacijų suformuojame klasių numerius
    class_label = np.array([all_beats[symbol] for symbol in atr_symbol])
    # print(class_label[:10])

    # Apskaičiuojame ir išvedame pasiskirstymą per klases
    cols = ['N','S','V','U']
    (unique, counts) = np.unique(class_label, return_counts=True)
    total = counts.sum()
    labels = [cols[u] for u in unique]    
    print("class labels: ", labels, counts, total)

    # cols_pattern, counts, total = get_freq_unique_values(class_label, cols_pattern=cols)
    # print("class labels: ", cols, counts, total)


    for beat_label in lst_beat_labels:

        df_beats_attr = create_selected_beats_df(SubjCode, atr_symbol, atr_sample, wl_side_beats, wr_side_beats, side_ext, beat_label)
        if (df_beats_attr.empty):
            print(f"Pūpsnių {beat_label} nerasta")
        else:    
            print(f"Rasta {len(df_beats_attr)} {beat_label} pūpsnių")
            # print(df_beats_attr.head())
        
        show_selected_beats(signal, atr_symbol, atr_sample, df_beats_attr, images_dir)



Bendras aplankas vaizdams: saved_images_of_selected_annotations
Max sekų vaizdų skaičius iš kiekvienos klasės: 20
Ieškomi pūpsniai: ['S']
Nagrinėjamų įrašų sąrašas: [10010, 10020, 10021]

SubjCode: 10010  file_name: 1621694.321 userId: 60a917b354352a3df86dc1f2 recId: 60a92aa454352a03116dc1f5
class labels:  ['N', 'V'] [6009    9] 6018
Pūpsnių S nerasta

SubjCode: 10020  file_name: 1625408.182 userId: 60e1d80f93b55b41529e9eaa recId: 60e1db5a93b55be9569e9eca
class labels:  ['N', 'S', 'V'] [1166    2   17] 1185
Rasta 2 S pūpsnių
file_path:  D:\DI\DUOM_2022_RUDUO_2\records_npy\saved_images_of_selected_annotations\S_beats\1625408.182_10020_72233_S.png
file_path:  D:\DI\DUOM_2022_RUDUO_2\records_npy\saved_images_of_selected_annotations\S_beats\1625408.182_10020_72473_S.png

SubjCode: 10021  file_name: 1625402.027 userId: 60e1d80f93b55b41529e9eaa recId: 60e1db5a93b55b56919e9ec0
class labels:  ['N', 'S', 'V'] [748   4  13] 765
Rasta 4 S pūpsnių
file_path:  D:\DI\DUOM_2022_RUDUO_2\records_npy\s

In [4]:
# Testavimui
np.set_printoptions(threshold=sys.maxsize)

# Nagrinėjamų pacientų kodai
# SubjCodes = [10000,10001,10002,10010,10011,10012,10013,10014]
SubjCodes = [10091, 10092]

# Variantas: testinė imtis  
# file_path = Path(rec_dir, 'all_subjcode_lst_z.csv')
# SubjCodes = list(np.loadtxt(file_path, delimiter=',', dtype="int"))

for SubjCode in SubjCodes:

    # Surandame ir išvedame paciento ir įrašo kodus, EKG reikšmių kiekį
    userNr, recNr = split_SubjCode(SubjCode)
    userId, recId, file_name = get_recId(rec_dir, userNr, recNr)
    print(f"\nSubjCode: {SubjCode}  file_name: {file_name} userId: {userId} recId: {recId}")

#  Bandymas surasti anotaciju 'U' indeksus
# 
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)
    
#  https://www.geeksforgeeks.org/python-get-the-indices-of-all-occurrences-of-an-element-in-a-list/
# indices = numpy.where(my_list == 1)[0]
    indices = np.where(atr_symbol == 'U')[0]

    # indices = [ind for ind, ele in enumerate(my_list) if ele == 1]

# https://www.pythonpool.com/check-if-numpy-array-is-empty/

    flag = np.size(indices)
    if (flag):
        print(indices)
        atr_symbol[indices] = 'Z'
        for i in indices:
            print(i, atr_symbol[i])
        print(atr_symbol)
    else:
        print('Tuscia')


SubjCode: 10091  file_name: 1631141.764 userId: 613b1d673d08d4d1f3cdc8f8 recId: 613b22d53d08d4fe94cdc9ed
Tuscia

SubjCode: 10092  file_name: 1631139.883 userId: 613b1d673d08d4d1f3cdc8f8 recId: 613b22d53d08d40ad7cdc9e7
[535 538 637 639 640]
535 Z
538 Z
637 Z
639 Z
640 Z
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'S' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N'
 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'S' 'N' 'N' 'N' 'N' 'N'